# Worksheet 2 Part 2 - DDP for Two Heated Tanks

This notebook implements Differential Dynamic Programming (DDP) for the two heated tanks system.

**Key differences from Part 1:**
- Part 1 used binary control (ON/OFF) with P-controller and LQR
- Part 2 uses continuous control with DDP for optimal trajectory generation

**Tasks:**
- Implement DDP algorithm for the two heated tanks
- Compare with LQR results from Part 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import bisect as bi

## Helper Functions for Grid Operations

In [ ]:
def edInt(a, s, b):
    """Create evenly discretized interval from a to b with step s"""
    return np.linspace(a, b, int(np.floor((b-a)/s)) + 1)

def idx(aList, val):
    """Find nearest index in sorted list for given value"""
    i = bi.bisect(aList, val)  # right insertion point
    if i <= 0:  # val is before first element
        return 0
    elif i >= len(aList):  # val is after last element
        return len(aList) - 1
    elif val - aList[i-1] < aList[i] - val:  # closer to left
        return i - 1
    else:  # closer to right
        return i

def lookup(aList, vals):
    """Find nearest indices for values (nearest-neighbor interpolation)"""
    if isinstance(vals, (int, float, np.floating)):
        vals = [vals]
    return np.array([idx(aList, v) for v in vals], dtype=np.int32)

## System Parameters (Mode 1: ON1 ∧ OFF2)

Same parameters as TP2 Part 1:

$$\dot{T}_1 = -a_1 T_1 + b_1 T_2 + h_1 u_1$$
$$\dot{T}_2 = b_2 T_1 - a_2 T_2 + h_2 u_2$$

In [ ]:
# System parameters (from TP2 Part 1)
a1 = 0.01   # Heat loss coefficient tank 1
a2 = 0.01   # Heat loss coefficient tank 2
b1 = 0.005  # Coupling coefficient 2→1
b2 = 0.005  # Coupling coefficient 1→2
h1 = 2.0    # Heater gain tank 1
h2 = 2.0    # Heater gain tank 2

print(f"System parameters:")
print(f"a1={a1}, a2={a2}, b1={b1}, b2={b2}, h1={h1}, h2={h2}")

## DDP Configuration

In [ ]:
# Time horizon
t0 = 0
dt = 0.1     # Time step (smaller for better accuracy)
tf = 100     # Final time
T = edInt(t0, dt, tf)
N = T.size

# State space discretization (temperature ranges)
dT1 = 1.0    # Temperature step for tank 1 (finer grid)
dT2 = 1.0    # Temperature step for tank 2
T1_min, T1_max = 0, 60    # Tank 1 temperature range °C
T2_min, T2_max = 0, 60    # Tank 2 temperature range °C

# Control space discretization
du1 = 0.25   # Control step for heater 1
du2 = 0.25   # Control step for heater 2
u1_min, u1_max = 0, 3     # Control range for heater 1
u2_min, u2_max = 0, 3     # Control range for heater 2

# Create admissible sets
T1_grid = edInt(T1_min, dT1, T1_max)
T2_grid = edInt(T2_min, dT2, T2_max)
U1 = edInt(u1_min, du1, u1_max)
U2 = edInt(u2_min, du2, u2_max)

# Create state space meshgrid
T1_mesh, T2_mesh = np.meshgrid(T1_grid, T2_grid)

# Initial and reference states (same as TP2 Part 1)
x0 = np.array([0.0, 50.0])   # Initial temperatures [T1, T2]
x_ref = np.array([25.0, 30.0])  # Reference temperatures [T1, T2]

# Penalty for inadmissible states
infStValInc = 1e6
nullCtr = 0.0

print(f"Time horizon: {t0} to {tf} s with dt={dt} s ({N} steps)")
print(f"State space: T1 ∈ [{T1_min}, {T1_max}]°C (step {dT1}), T2 ∈ [{T2_min}, {T2_max}]°C (step {dT2})")
print(f"State grid sizes: {T1_grid.size} x {T2_grid.size} = {T1_grid.size * T2_grid.size} states")
print(f"Control space: u1 ∈ [{u1_min}, {u1_max}] (step {du1}), u2 ∈ [{u2_min}, {u2_max}] (step {du2})")
print(f"Control grid sizes: {U1.size} x {U2.size} = {U1.size * U2.size} control combinations")
print(f"Initial state: T1={x0[0]}°C, T2={x0[1]}°C")
print(f"Reference state: T1={x_ref[0]}°C, T2={x_ref[1]}°C")

## Cost Function

$$L(x, u) = Q_1(T_1 - T_1^{ref})^2 + Q_2(T_2 - T_2^{ref})^2 + R_1 u_1^2 + R_2 u_2^2$$

In [ ]:
# Cost matrices (similar to LQR weights from Part 1)
Q1 = 10.0    # State cost weight for tank 1
Q2 = 10.0    # State cost weight for tank 2
R1 = 1.0     # Control cost weight for heater 1
R2 = 1.0     # Control cost weight for heater 2

print(f"Cost weights: Q1={Q1}, Q2={Q2}, R1={R1}, R2={R2}")

## Initialize Value Function and Optimal Control Arrays

In [ ]:
# Initialize arrays
# V[i, j, k] = optimal cost-to-go from state (T1_grid[j], T2_grid[i]) at time k
# u1_opt[i, j, k] = optimal control u1 at state (T1_grid[j], T2_grid[i]) at time k
# u2_opt[i, j, k] = optimal control u2 at state (T1_grid[j], T2_grid[i]) at time k
V = np.zeros((T2_grid.size, T1_grid.size, N))
u1_opt = np.zeros_like(V)
u2_opt = np.zeros_like(V)

# Terminal cost (at time N-1)
V[:, :, N-1] = Q1 * (T1_mesh - x_ref[0])**2 + Q2 * (T2_mesh - x_ref[1])**2

print(f"Initialized V with shape: {V.shape}")
print(f"Terminal cost V[:,:,{N-1}] set based on reference state")

## DDP Backward Pass (Value Iteration)

Bellman equation:
$$V(x, k) = \min_{u_1, u_2} \left[ L(x, u) + V(f(x, u), k+1) \right]$$

where $f(x,u)$ is the next state given by the system dynamics.

In [ ]:
print("Starting DDP backward pass...")
print(f"Processing {N-1} time steps x {T2_grid.size} x {T1_grid.size} states x {U1.size} x {U2.size} controls")
print(f"Total iterations: {(N-1) * T2_grid.size * T1_grid.size}")
print("This may take a minute...\n")

depth = 0  # Start from beginning of horizon
inadmissible_count = 0

for k in range(N-2, depth-1, -1):  # Go backwards from N-2 to 0
    if k % 100 == 0:
        print(f"Processing time step {k}/{N-2}...")
    
    for i in range(T2_grid.size):
        for j in range(T1_grid.size):
            # Current state
            T1_curr = T1_grid[j]
            T2_curr = T2_grid[i]
            
            # Initialize minimum cost
            min_cost = float('inf')
            best_u1 = nullCtr
            best_u2 = nullCtr
            
            # Try all control combinations
            for u1_val in U1:
                for u2_val in U2:
                    # Instant cost
                    L = Q1 * (T1_curr - x_ref[0])**2 + Q2 * (T2_curr - x_ref[1])**2 + \
                        R1 * u1_val**2 + R2 * u2_val**2
                    
                    # Predict next state using mode 1 dynamics
                    T1_next = T1_curr + (-a1 * T1_curr + b1 * T2_curr + h1 * u1_val) * dt
                    T2_next = T2_curr + (b2 * T1_curr - a2 * T2_curr + h2 * u2_val) * dt
                    
                    # Check if next state is admissible
                    if (T1_next >= T1_min and T1_next <= T1_max and 
                        T2_next >= T2_min and T2_next <= T2_max):
                        
                        # Find nearest grid indices (nearest-neighbor interpolation)
                        j_next = lookup(T1_grid, T1_next)[0]
                        i_next = lookup(T2_grid, T2_next)[0]
                        
                        # Bellman equation: total cost = instant cost + future cost
                        total_cost = L + V[i_next, j_next, k+1]
                        
                        # Update if this is better
                        if total_cost < min_cost:
                            min_cost = total_cost
                            best_u1 = u1_val
                            best_u2 = u2_val
            
            # Store optimal cost and control
            if min_cost == float('inf'):
                # No admissible control found - penalize
                V[i, j, k] = V[i, j, k+1] + infStValInc
                inadmissible_count += 1
            else:
                V[i, j, k] = min_cost
            
            u1_opt[i, j, k] = best_u1
            u2_opt[i, j, k] = best_u2

print(f"\nDDP backward pass complete!")
print(f"Inadmissible state-time points: {inadmissible_count} / {(N-1) * T2_grid.size * T1_grid.size}")

## Forward Simulation Using Optimal Policy

In [ ]:
# Simulate from initial state
x = np.zeros((N, 2))
x[0, :] = x0
u1_sim = np.zeros(N)
u2_sim = np.zeros(N)

print(f"Starting forward simulation from x0 = {x0}")

for k in range(N-1):
    # Check if current state is admissible
    if not (x[k, 0] >= T1_min and x[k, 0] <= T1_max and
            x[k, 1] >= T2_min and x[k, 1] <= T2_max):
        print(f"Warning: Stopped at k={k}/{N}: state [{x[k,0]:.2f}, {x[k,1]:.2f}] out of bounds")
        x = x[:k+1, :]
        u1_sim = u1_sim[:k+1]
        u2_sim = u2_sim[:k+1]
        T = T[:k+1]
        break
    
    # Find nearest grid indices
    j_T1 = lookup(T1_grid, x[k, 0])[0]
    i_T2 = lookup(T2_grid, x[k, 1])[0]
    
    # Get optimal control
    u1_sim[k] = u1_opt[i_T2, j_T1, k]
    u2_sim[k] = u2_opt[i_T2, j_T1, k]
    
    # Propagate dynamics
    x[k+1, 0] = x[k, 0] + (-a1 * x[k, 0] + b1 * x[k, 1] + h1 * u1_sim[k]) * dt
    x[k+1, 1] = x[k, 1] + (b2 * x[k, 0] - a2 * x[k, 1] + h2 * u2_sim[k]) * dt

print(f"Simulation complete!")
print(f"Final state: T1={x[-1,0]:.2f}°C, T2={x[-1,1]:.2f}°C")
print(f"Reference: T1={x_ref[0]}°C, T2={x_ref[1]}°C")
print(f"Final error: T1={x[-1,0]-x_ref[0]:.2f}°C, T2={x[-1,1]-x_ref[1]:.2f}°C")

## Visualization

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(18, 12))

# State trajectories
ax1 = plt.subplot(3, 3, 1)
ax1.plot(T[:len(x)], x[:, 0], 'b-', linewidth=2, label='Tank 1 (DDP)')
ax1.axhline(y=x_ref[0], color='red', linestyle='--', label=f'Reference ({x_ref[0]}°C)')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Temperature [°C]')
ax1.set_title('Tank 1 Temperature Trajectory')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = plt.subplot(3, 3, 2)
ax2.plot(T[:len(x)], x[:, 1], 'orange', linewidth=2, label='Tank 2 (DDP)')
ax2.axhline(y=x_ref[1], color='red', linestyle='--', label=f'Reference ({x_ref[1]}°C)')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Temperature [°C]')
ax2.set_title('Tank 2 Temperature Trajectory')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Control signals
ax3 = plt.subplot(3, 3, 4)
ax3.plot(T[:len(u1_sim)], u1_sim, 'g-', linewidth=2, label='u1 (Heater 1)')
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Control Signal')
ax3.set_title('Optimal Control - Heater 1')
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4 = plt.subplot(3, 3, 5)
ax4.plot(T[:len(u2_sim)], u2_sim, 'm-', linewidth=2, label='u2 (Heater 2)')
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('Control Signal')
ax4.set_title('Optimal Control - Heater 2')
ax4.legend()
ax4.grid(True, alpha=0.3)

# State space trajectory
ax5 = plt.subplot(3, 3, 3)
ax5.plot(x[:, 0], x[:, 1], 'b-', linewidth=2, alpha=0.7)
ax5.plot(x[0, 0], x[0, 1], 'go', markersize=10, label='Initial')
ax5.plot(x[-1, 0], x[-1, 1], 'ro', markersize=10, label='Final')
ax5.plot(x_ref[0], x_ref[1], 'r*', markersize=15, label='Reference')
ax5.set_xlabel('T1 [°C]')
ax5.set_ylabel('T2 [°C]')
ax5.set_title('State Space Trajectory')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Value function at t=0
ax6 = plt.subplot(3, 3, 6, projection='3d')
ax6.plot_surface(T1_mesh, T2_mesh, V[:, :, 0], cmap='viridis', alpha=0.8)
# Mark initial state value
j_x0 = lookup(T1_grid, x0[0])[0]
i_x0 = lookup(T2_grid, x0[1])[0]
V_x0 = V[i_x0, j_x0, 0]
ax6.plot([x0[0]], [x0[1]], [V_x0], 'r*', markersize=15)
ax6.set_xlabel('T1 [°C]')
ax6.set_ylabel('T2 [°C]')
ax6.set_zlabel('V(x, t=0)')
ax6.set_title(f'Value Function at t=0\nV(x0)={V_x0:.2f}')

# Value function at terminal time
ax7 = plt.subplot(3, 3, 9, projection='3d')
ax7.plot_surface(T1_mesh, T2_mesh, V[:, :, N-1], cmap='plasma', alpha=0.8)
ax7.set_xlabel('T1 [°C]')
ax7.set_ylabel('T2 [°C]')
ax7.set_zlabel('V(x, t=tf)')
ax7.set_title('Terminal Cost (t=tf)')

# Optimal control policy u1 at t=0
ax8 = plt.subplot(3, 3, 7, projection='3d')
ax8.plot_surface(T1_mesh, T2_mesh, u1_opt[:, :, 0], cmap='coolwarm', alpha=0.8)
ax8.set_xlabel('T1 [°C]')
ax8.set_ylabel('T2 [°C]')
ax8.set_zlabel('u1*')
ax8.set_title('Optimal Control u1 at t=0')

# Optimal control policy u2 at t=0
ax9 = plt.subplot(3, 3, 8, projection='3d')
ax9.plot_surface(T1_mesh, T2_mesh, u2_opt[:, :, 0], cmap='coolwarm', alpha=0.8)
ax9.set_xlabel('T1 [°C]')
ax9.set_ylabel('T2 [°C]')
ax9.set_zlabel('u2*')
ax9.set_title('Optimal Control u2 at t=0')

plt.tight_layout()
plt.show()

## Performance Metrics

In [ ]:
# Calculate performance metrics
ss_error_T1 = x[-1, 0] - x_ref[0]
ss_error_T2 = x[-1, 1] - x_ref[1]

# Calculate total cost
total_cost = 0
for k in range(len(x)-1):
    L_k = Q1 * (x[k, 0] - x_ref[0])**2 + Q2 * (x[k, 1] - x_ref[1])**2 + \
          R1 * u1_sim[k]**2 + R2 * u2_sim[k]**2
    total_cost += L_k * dt

# Terminal cost
L_N = Q1 * (x[-1, 0] - x_ref[0])**2 + Q2 * (x[-1, 1] - x_ref[1])**2
total_cost += L_N

print("\n" + "="*60)
print("PERFORMANCE METRICS - DDP")
print("="*60)
print(f"Steady-state error Tank 1: {ss_error_T1:.4f} °C")
print(f"Steady-state error Tank 2: {ss_error_T2:.4f} °C")
print(f"Total accumulated cost: {total_cost:.2f}")
print(f"Optimal value at x0: {V_x0:.2f}")
print(f"\nAverage control effort:")
print(f"  u1: {np.mean(u1_sim):.4f} (max: {np.max(u1_sim):.4f})")
print(f"  u2: {np.mean(u2_sim):.4f} (max: {np.max(u2_sim):.4f})")
print("="*60)

## Comparison with LQR (from Part 1)

**Key Differences:**

1. **DDP (this notebook):**
   - Finite-horizon optimal control
   - Time-varying optimal policy
   - Can handle constraints naturally
   - Computationally expensive (backward pass)

2. **LQR (Part 1):**
   - Infinite-horizon optimal control
   - Time-invariant feedback gain
   - Analytical solution
   - Computationally efficient

**When to use DDP:**
- Finite time horizon problems
- Time-varying systems or costs
- State/control constraints
- Trajectory optimization

**When to use LQR:**
- Regulation around equilibrium
- Real-time control (fast)
- Linear systems with quadratic cost
- No hard constraints